<a href="https://colab.research.google.com/github/Akshat13shah/data_augmentation_and_modeltraining/blob/main/Model_Training_Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CS203 Lab Assignment 5

### Team Number: 18
* Name: Paras Prashant Shirvale
* Roll No: 23110232
---
* Name: Akshat Shah
* Roll No: 23110293
---

### Task 2: Model Training
*Note: Initial weights of the model should be the same when training with both datasets.*

1. Choose (microsoft/resnet-50)model from the hugging face and initialize its new weights.
2. Train model(created in the above point) on a downloaded dataset, without augmentation.
3. Train model(created in the first point) on a downloaded dataset with augmentation.
4. Get the precision, recall, F1 score, and accuracy of both the models on the test set.


In [202]:
from transformers import AutoImageProcessor, ResNetConfig, ResNetForImageClassification
import torch
import os
import random
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn as nn

In [203]:
# Dataset paths
path = "/content/drive/MyDrive/test"

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard ResNet normalization
])

In [204]:
# Load dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

cats = dataset.imgs[:70]
dogs = dataset.imgs[70:]

train = cats[:56] + dogs[:56]
test = cats[56:] + dogs[56:]

print(len(train),len(test))

random.shuffle(train)

112 28


In [205]:
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, img_list, transform=None):
        self.img_list = img_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path, label = self.img_list[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label

# Assuming 'train' and 'test' lists are already created and shuffled
train_dataset = CustomDataset(train, transform=transform)
test_dataset = CustomDataset(test, transform=transform)

# Create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Example of using the DataLoader
for images, labels in train_loader:
    print(images.shape, labels.shape)


torch.Size([32, 3, 224, 224]) torch.Size([32])
torch.Size([32, 3, 224, 224]) torch.Size([32])
torch.Size([32, 3, 224, 224]) torch.Size([32])
torch.Size([16, 3, 224, 224]) torch.Size([16])


In [206]:
image1, lbl1 = train_dataset[0]
image2, lbl2 = train_dataset[1]

In [207]:
lbl1

1

In [208]:
classes = dataset.classes

In [209]:
train_dataloader = DataLoader(train, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test, batch_size=16, shuffle=True)

In [210]:
train

[('/content/drive/MyDrive/test/dogs/dog_369.jpg', 1),
 ('/content/drive/MyDrive/test/dogs/dog_150.jpg', 1),
 ('/content/drive/MyDrive/test/dogs/dog_147.jpg', 1),
 ('/content/drive/MyDrive/test/dogs/dog_44.jpg', 1),
 ('/content/drive/MyDrive/test/cats/cat_113.jpg', 0),
 ('/content/drive/MyDrive/test/cats/cat_251.jpg', 0),
 ('/content/drive/MyDrive/test/dogs/dog_181.jpg', 1),
 ('/content/drive/MyDrive/test/dogs/dog_443.jpg', 1),
 ('/content/drive/MyDrive/test/dogs/dog_130.jpg', 1),
 ('/content/drive/MyDrive/test/cats/cat_384.jpg', 0),
 ('/content/drive/MyDrive/test/dogs/dog_43.jpg', 1),
 ('/content/drive/MyDrive/test/cats/cat_5.jpg', 0),
 ('/content/drive/MyDrive/test/cats/cat_355.jpg', 0),
 ('/content/drive/MyDrive/test/cats/cat_223.jpg', 0),
 ('/content/drive/MyDrive/test/cats/cat_468.jpg', 0),
 ('/content/drive/MyDrive/test/dogs/dog_159.jpg', 1),
 ('/content/drive/MyDrive/test/cats/cat_124.jpg', 0),
 ('/content/drive/MyDrive/test/cats/cat_545.jpg', 0),
 ('/content/drive/MyDrive/test/c

In [211]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [212]:
# Load the model configuration and set it up with random weights initialization
config = ResNetConfig.from_pretrained("microsoft/resnet-50")
config.num_labels = len(classes)

# Initialize the model with the custom configuration (without pretrained weights)
model = ResNetForImageClassification(config)

# Initialize weights randomly using a custom function if needed
def init_weights(module):
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        torch.nn.init.xavier_normal_(module.weight)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)

model.apply(init_weights)


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [213]:
model.config

ResNetConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "ResNetForImageClassification"
  ],
  "depths": [
    3,
    4,
    6,
    3
  ],
  "downsample_in_bottleneck": false,
  "downsample_in_first_stage": false,
  "embedding_size": 64,
  "hidden_act": "relu",
  "hidden_sizes": [
    256,
    512,
    1024,
    2048
  ],
  "layer_type": "bottleneck",
  "model_type": "resnet",
  "num_channels": 3,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4"
  ],
  "torch_dtype": "float32",
  "transformers_version": "4.48.2"
}

In [214]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [215]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_model(test_loader, model, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")


# Function to train the model
def train_model(train_loader, model, device, num_epochs=5):
    model.to(device)  # Move the model to the device (GPU or CPU)
    model.train()  # Set the model to training mode

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Use Adam optimizer
    criterion = torch.nn.CrossEntropyLoss()  # Use CrossEntropy loss for classification

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Move both inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)
            model.train()
            optimizer.zero_grad()  # Zero the parameter gradients

            # Forward pass
            outputs = model(inputs).logits  # Forward pass through the model
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagation

            optimizer.step()  # Update model parameters

            running_loss += loss.item()  # Track the loss

        # Print the loss for this epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
        evaluate_model(test_loader, model, device)



# # Function to evaluate the model on the test set
# def evaluate_model(test_loader, model, device):
#     model.to(device)
#     model.eval()

#     all_preds = []
#     all_labels = []

#     with torch.no_grad():
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)

#             outputs = model(inputs).logits
#             _, preds = torch.max(outputs, 1)

#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     # Calculate precision, recall, F1 score, and accuracy
#     precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
#     accuracy = accuracy_score(all_labels, all_preds)

#     return precision, recall, f1, accuracy



In [216]:
# Step 8: Train the model
train_model(train_loader, model, device, num_epochs=50)

# Evaluate the model on the test set
evaluate_model(test_loader, model, device)

Epoch 1/50, Loss: 0.9791367799043655
Test Accuracy: 50.0%
Epoch 2/50, Loss: 0.8290966302156448
Test Accuracy: 50.0%
Epoch 3/50, Loss: 0.6671561449766159
Test Accuracy: 50.0%
Epoch 4/50, Loss: 0.6359817385673523
Test Accuracy: 50.0%
Epoch 5/50, Loss: 0.5589600652456284
Test Accuracy: 50.0%
Epoch 6/50, Loss: 0.5388449281454086
Test Accuracy: 50.0%
Epoch 7/50, Loss: 0.4421068951487541
Test Accuracy: 50.0%
Epoch 8/50, Loss: 0.44373198598623276
Test Accuracy: 50.0%
Epoch 9/50, Loss: 0.4065786972641945
Test Accuracy: 50.0%
Epoch 10/50, Loss: 0.34124646335840225
Test Accuracy: 50.0%
Epoch 11/50, Loss: 0.2908700592815876
Test Accuracy: 46.42857142857143%
Epoch 12/50, Loss: 0.1933519933372736
Test Accuracy: 46.42857142857143%
Epoch 13/50, Loss: 0.17878206074237823
Test Accuracy: 53.57142857142857%
Epoch 14/50, Loss: 0.20554085820913315
Test Accuracy: 60.714285714285715%
Epoch 15/50, Loss: 0.11323683615773916
Test Accuracy: 53.57142857142857%
Epoch 16/50, Loss: 0.2834768835455179
Test Accuracy: 

In [218]:
evaluate_model(train_loader, model, device)

Test Accuracy: 98.21428571428571%


In [ ]:
# Check if CUDA is available and select the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train and evaluate on the dataset without augmentation
print("Training without augmentation:")
train_model(train_dataloader, model, device, num_epochs=5)
precision, recall, f1, accuracy = evaluate_model(test_dataloader, model, device)
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}, Accuracy: {accuracy}")

# # Reinitialize the model (same random weights) for training with augmentation
# model.apply(init_weights)  # Reinitialize to maintain same random weights

# # Train and evaluate on the dataset with augmentation
# print("Training with augmentation:")
# train_model(train_loader_with_augmentation, model, device, num_epochs=5)
# precision, recall, f1, accuracy = evaluate_model(test_loader, model, device)
# print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}, Accuracy: {accuracy}")


In [ ]:
def init_weights(module):
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        torch.nn.init.xavier_normal_(module.weight)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)

In [ ]:
# # Step 1: Set a fixed random seed for reproducibility
# seed = 42
# torch.manual_seed(seed)  # For PyTorch
# np.random.seed(seed)     # For NumPy
# torch.cuda.manual_seed(seed)  # If using GPU
# torch.cuda.manual_seed_all(seed)  # If using multiple GPUs
# torch.backends.cudnn.deterministic = True  # Ensures deterministic results
# torch.backends.cudnn.benchmark = False   # Disable cudnn auto-tuner for consistency

In [ ]:
# Step 2: Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Step 3: Load ResNet-50 from Hugging Face
model_name = "microsoft/resnet-50"
model = ResNetForImageClassification.from_pretrained(model_name)
config = model.config

In [ ]:
# Step 4: Initialize the model's weights with new random weights
model.apply(init_weights)

In [ ]:
model.config

In [ ]:
# Image for the Architecture

In [ ]:
# Step 6: Dataset paths and Image preprocessing (No Augmentation)


In [ ]:
# Step 7: Set up the training loop
def train_model(train_loader, model, device, num_epochs=5):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs).logits  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model parameters

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

In [ ]:
# Step 8: Train the model
train_model(train_loader, model, device, num_epochs=5)

In [ ]:
# Step 9: Evaluate the model on the test set (optional)
def evaluate_model(test_loader, model, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")